# Merging LandCover.ai and DeepGlobe dataset

### todo
- [ ] find fitting resolution
- [ ] transfer labels to same
- [ q] Resize resolution
- [ ] v0 verwenden weil da keine straßen als label sind??

### additional info

using landcover.ai v1

In [117]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [118]:
# data root
LANDCOVER_AI_ROOT = '../data/landcover.ai.v1'
DEEPGLOBE_ROOT = '../data/deepglobe'
MERGED_ROOT = '../data/merge'

In [ ]:
from PIL import Image

def map_class_to_color(image_path, output_path):
    # Open the image
    img = Image.open(image_path)

    # Get the width and height of the image
    width, height = img.size

    turquoise = (0, 255, 255)
    green = (0, 255, 0)
    blue = (0, 0, 255)
    black = (0, 0, 0)
    building = 1
    woodland = 2
    water = 3
    road = 4

    # Iterate over each pixel in the image
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            r, g, b = img.getpixel((x, y))

            # Check if the pixel values fall within the specified range (1 to 5)
            if 1 <= r <= 5 and 1 <= g <= 5 and 1 <= b <= 5:
                if r == g and g == b:
                    if r == building:
                        img.putpixel((x, y), turquoise)
                    elif r == woodland:
                        img.putpixel((x, y), green)
                    elif r == water:
                        img.putpixel((x, y), blue)
                    elif r == road:
                        img.putpixel((x, y), black)
                    else:
                        print(f"not a valid class. Pixel: ({r},{g},{b})")
                        return
                else:
                    print(f"not a number between 1 and 5. Pixel: ({r},{g},{b})")
                    return

    # Save the modified image
    img.save(output_path)


# Example usage:
input_image_path = "C:/Users/Uni/Documents/DLIVP/dlivp-workspace/data/merge/N-33-130-A-d-4-4_214_m.png"
output_image_path = "../data/N-33-130-A-d-4-4_214_m.png"

modify_image(input_image_path, output_image_path)

In [126]:
from PIL import Image
import os

def check_pixel_size(folder_path):
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff'))]

    if not image_files:
        print("No image files found in the specified folder.")
        return

    first_image_path = os.path.join(folder_path, image_files[0])
    with Image.open(first_image_path) as reference_image:
        reference_size = reference_image.size
        
    globalSize= 0
    
    for image_file in image_files[1:]:
        current_image_path = os.path.join(folder_path, image_file)
        with Image.open(current_image_path) as current_image:
            globalSize = current_image.size
            if current_image.size != reference_size:
                print(f"Pixel size of {current_image_path} is different from the reference image. {current_image.size} doesn't match {reference_size}")
                return

    print(f"All images have the same pixel size of {globalSize}.")
    print(f"Amount of pictures {len(image_files)}.")



In [120]:
import os
import shutil
import re

def copy_files_with_word(source_dir, destination_dir, keyword):
    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Iterate over files in the source directory
    for filename in os.listdir(source_dir):
        source_path = os.path.join(source_dir, filename)

        # Check if the file contains the specified keyword
        if os.path.isfile(source_path) and keyword in filename:
            newFilename = re.sub(r'(_sat|_mask)', '', filename)
            destination_path = os.path.join(destination_dir, newFilename)

            # Copy the file to the destination directory
            shutil.copyfile(source_path, destination_path)
            print(f"Copied: {filename} to {newFilename}")

In [122]:
#!/usr/bin/env python3

import glob
import os

import cv2

def split_images(dir):
    IMGS_DIR = dir + "/images"
    MASKS_DIR = dir + "/masks"
    TARGET_SIZE = 512
    OUTPUT_DIR = "../data/merge"
    
    if "landcover.ai" in dir:
        img_paths = glob.glob(os.path.join(IMGS_DIR, "*.tif"))
        mask_paths = glob.glob(os.path.join(MASKS_DIR, "*.tif"))
        img_paths.sort()
        mask_paths.sort()
    
    if "deepglobe" in dir:
        copy_files_with_word(dir, IMGS_DIR, "sat")
        copy_files_with_word(dir, MASKS_DIR, "mask")
        img_paths = glob.glob(os.path.join(IMGS_DIR, "*.jpg"))
        mask_paths = glob.glob(os.path.join(MASKS_DIR, "*.png"))
        img_paths.sort()
        mask_paths.sort()
        
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        
    for i, (img_path, mask_path) in enumerate(zip(img_paths, mask_paths)):
        img_filename = os.path.splitext(os.path.basename(img_path))[0]
        mask_filename = os.path.splitext(os.path.basename(mask_path))[0]
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path)
    
        assert img_filename == mask_filename and img.shape[:2] == mask.shape[:2]
    
        k = 0
        for y in range(0, img.shape[0], TARGET_SIZE):
            for x in range(0, img.shape[1], TARGET_SIZE):
                img_tile = img[y:y + TARGET_SIZE, x:x + TARGET_SIZE]
                mask_tile = mask[y:y + TARGET_SIZE, x:x + TARGET_SIZE]
    
                if img_tile.shape[0] == TARGET_SIZE and img_tile.shape[1] == TARGET_SIZE:
                    out_img_path = os.path.join(OUTPUT_DIR, "{}_{}.jpg".format(img_filename, k))
                    cv2.imwrite(out_img_path, img_tile)
    
                    out_mask_path = os.path.join(OUTPUT_DIR, "{}_{}_m.png".format(mask_filename, k))
                    cv2.imwrite(out_mask_path, mask_tile)
    
                k += 1
    
        print("Processed {} {}/{}".format(img_filename, i + 1, len(img_paths)))
    check_pixel_size(OUTPUT_DIR)


In [123]:
# cropping images due to gpu resource and the need of fixed size for the algorithm

split_images(LANDCOVER_AI_ROOT)
split_images(DEEPGLOBE_ROOT + '/train')

Processed M-33-20-D-c-4-2 1/41
Processed M-33-20-D-d-3-3 2/41
Processed M-33-32-B-b-4-4 3/41
Processed M-33-48-A-c-4-4 4/41
Processed M-33-7-A-d-2-3 5/41
Processed M-33-7-A-d-3-2 6/41
Processed M-34-32-B-a-4-3 7/41
Processed M-34-32-B-b-1-3 8/41
Processed M-34-5-D-d-4-2 9/41
Processed M-34-51-C-b-2-1 10/41
Processed M-34-51-C-d-4-1 11/41
Processed M-34-55-B-b-4-1 12/41
Processed M-34-56-A-b-1-4 13/41
Processed M-34-6-A-d-2-2 14/41
Processed M-34-65-D-a-4-4 15/41
Processed M-34-65-D-c-4-2 16/41
Processed M-34-65-D-d-4-1 17/41
Processed M-34-68-B-a-1-3 18/41
Processed M-34-77-B-c-2-3 19/41
Processed N-33-104-A-c-1-1 20/41
Processed N-33-119-C-c-3-3 21/41
Processed N-33-130-A-d-3-3 22/41
Processed N-33-130-A-d-4-4 23/41
Processed N-33-139-C-d-2-2 24/41
Processed N-33-139-C-d-2-4 25/41
Processed N-33-139-D-c-1-3 26/41
Processed N-33-60-D-c-4-2 27/41
Processed N-33-60-D-d-1-2 28/41
Processed N-33-96-D-d-1-1 29/41
Processed N-34-106-A-b-3-4 30/41
Processed N-34-106-A-c-1-3 31/41
Processed N-

In [127]:
check_pixel_size(OUTPUT_DIR)

All images have the same pixel size of (512, 512).
Amount of pictures 47044.


In [128]:
def read_png_file(file_path):
    try:
        # Open the PNG file
        with Image.open(file_path) as img:
            # Get the pixel data
            pixel_data = list(img.getdata())
            width, height = img.size

            # You can now process or manipulate the pixel data as needed
            # For example, printing the RGB values of the first pixel
            print(f"RGB values of the first pixel: {pixel_data[0]}")

    except Exception as e:
        print(f"Error reading PNG file: {e}")

In [129]:
read_png_file('../data/merge/M-34-65-D-c-4-2_73_m.png')

RGB values of the first pixel: (2, 2, 2)


urban_land,0,255,255
agriculture_land,255,255,0
rangeland,255,0,255
forest_land,0,255,0
water,0,0,255
barren_land,255,255,255
unknown,0,0,0

building (1), woodland (2), water(3), road(4)
->
building 1 -> 0,255,255
woodland 2 -> 0,255,0
water    3 -> 0,0,255
road     4 -> 0,0,0
